## Imports:
* Pathlib for easy path handling
* nibabel as the files are provided in the NIfTI format
* numpy for data saving and processing
* matplotlib for plotting
* tqdm for nice progressing bars

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
%matplotlib notebook
from pathlib import Path
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
root = Path("/content/drive/MyDrive/Colab Notebooks/Task02_Heart/Task02_Heart/imagesTr")
label = Path("/content/drive/MyDrive/Colab Notebooks/Task02_Heart/Task02_Heart/labelsTr")

In [ ]:
def change_img_to_label_path(path):
    """ 
    Replaces imagesTr with labelsTr
    """
    parts = list(path.parts)  # get all directories within the path
    parts[parts.index("imagesTr")] = "labelsTr"  # Replace imagesTr with labelsTr
    return Path(*parts)  # Combine list back into a Path object


In [ ]:
sample_path = list(root.glob("la*"))[0]  
sample_path_label = change_img_to_label_path(sample_path)

In [ ]:
sample_path, sample_path_label

(PosixPath('/content/drive/MyDrive/Colab Notebooks/Task02_Heart/Task02_Heart/imagesTr/la_003.nii.gz'),
 PosixPath('/content/drive/MyDrive/Colab Notebooks/Task02_Heart/Task02_Heart/labelsTr/la_003.nii.gz'))

In [ ]:
data = nib.load(sample_path)
label = nib.load(sample_path_label)

mri = data.get_fdata()
mask = label.get_fdata().astype(np.uint8)  # Class labels should not be handled as float64

In [ ]:
nib.aff2axcodes(data.affine)

('R', 'A', 'S')



We used celluloid module (https://github.com/jwkvam/celluloid) for live visualization in matplotlib.<br />
Additionally we imported HTML from IPython.display to create a video of the volume.

In [ ]:
pip install celluloid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install IPython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from celluloid import Camera
from IPython.display import HTML

In [ ]:
fig = plt.figure()
camera = Camera(fig) 

for i in range(mri.shape[2]): 
    plt.imshow(mri[:,:,i], cmap="bone")
    mask_ = np.ma.masked_where(mask[:,:,i]==0, mask[:,:,i])
    plt.imshow(mask_, alpha=0.5, cmap="autumn")
    camera.snap()  
animation = camera.animate()  




<IPython.core.display.Javascript object>

In [ ]:
HTML(animation.to_html5_video())


In [ ]:
def normalize(full_volume):
    """
    Z-Normalization of the whole subject
    """
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized

def standardize(normalized_data):
    """
    Standardize the normalized data into the 0-1 range
    """
    standardized_data = (normalized_data - normalized_data.min()) / (normalized_data.max() - normalized_data.min())
    return standardized_data


In [ ]:
def normalize(full_volume):
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized

def standardize(normalized):
    standardized = (normalized - normalized.min()) / (normalized.max() - normalized.min())
    return standardized

Now we can loop over the subjects and store the preprocessed files in the corresponding directories.
To evaluate our model we use the last 4 subjects as validation set.

In [ ]:
all_files = list(root.glob("la*"))  
len(all_files)

20

In [ ]:
save_root = Path("Preprocessed")

for counter, path_to_mri_data in enumerate(tqdm(all_files)):
    
    path_to_label = change_img_to_label_path(path_to_mri_data)
    
    mri = nib.load(path_to_mri_data)
    assert nib.aff2axcodes(mri.affine) == ("R", "A", "S")    
    mri_data = mri.get_fdata()
    label_data = nib.load(path_to_label).get_fdata().astype(np.uint8)
    
    mri_data = mri_data[32:-32, 32:-32]
    label_data = label_data[32:-32, 32:-32]
    
    normalized_mri_data = normalize(mri_data)
    standardized_mri_data = standardize(normalized_mri_data)
    
    if counter < 17:
        current_path = save_root/"train"/str(counter)
    else:
        current_path = save_root/"val"/str(counter)
    
    for i in range(standardized_mri_data.shape[-1]):
        slice = standardized_mri_data[:,:,i]
        mask = label_data[:,:,i]
        slice_path = current_path/"data"
        mask_path = current_path/"masks"
        slice_path.mkdir(parents=True, exist_ok=True)
        mask_path.mkdir(parents=True, exist_ok=True)
        
        np.save(slice_path/str(i), slice)
        np.save(mask_path/str(i), mask)
        
        
    

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
path = Path("Preprocessed/train/1/") 


In [ ]:
file = "50.npy"
slice = np.load(path/"data"/file)
mask = np.load(path/"masks"/file)

In [ ]:
plt.figure()
plt.imshow(slice, cmap="bone")
mask_ = np.ma.masked_where(mask==0, mask)
plt.imshow(mask_, cmap="autumn")

<IPython.core.display.Javascript object>

In [ ]:
print(slice.min(), slice.max())

0.0 0.8387631143014908
